In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from socket import timeout
import http
import gc
import pandas as pd
import datetime
import csv
import numpy as np
import pyodbc
import time

In [2]:
import json
import pyodbc
#import numpy as np
server = 'masteksql.database.windows.net'
database = 'ProspectData'
username = 'yashpal'
password = 'Welcome498$'
driver= '{SQL Server}'

In [9]:
#BING SEARCH DATA
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

cursor.execute("Select ID,ProspectName,SearchResult from ProspectData WHERE BINGSEARCHUPDATES='TRUE' ") #AND ISPROCESSED='False'
urllist=[]
bunch=[]
pid=[]
row = cursor.fetchone()
while row:
    bunch.append(str(row[2]))
    for prospect in bunch:
        data=json.loads(prospect)
    for each in data:
        urllist.append(each['url'])
        pid.append(row[0])
    row = cursor.fetchone()

d=[]
e=[]
for c in zip(pid,urllist):
    if c[1] not in d:
        d.append(c[0])
        e.append(c[1])
#return zip(e,d)
f=zip(d,e)

In [10]:
len(e)

1280

In [13]:
e

['https://en.wikipedia.org/wiki/Capita',
 'https://www.bmj.com/content/362/bmj.k3228',
 'https://kittysjones.wordpress.com/2018/07/05/barnet-capita-contractor-on-fraud-charge/',
 'https://www.glassdoor.com/Reviews/Capita-Reviews-E10883.htm',
 'http://directorsclub.news/2018/07/04/capita-secures-contract-extension-with-npower/',
 'https://www.payscale.com/research/IN/Employer=Capita_india/Salary',
 'https://www.cnbc.com/2018/07/23/reuters-america-uk-mod-suspends-capita-contract-after-rival-serco-challenges-decision.html',
 'https://twitter.com/CapitaPS',
 'https://en.wikipedia.org/wiki/Public%E2%80%93private_partnership',
 'https://twitter.com/C2k_info',
 'https://www.pprune.org/military-aviation/610426-capita-wins-defence-fire-rescue.html',
 'https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=5491410',
 'https://www.glassdoor.com/Reviews/Capita-Pune-Reviews-EI_IE10883.0,6_IL.7,11_IM1072.htm',
 'https://www.payscale.com/research/UK/Employer=Capita_Resourcing/Salary

In [11]:
urls=[]
for a in e:
    if a not in urls:
        urls.append(a)

In [12]:
pd.DataFrame({"urls":urls}).to_csv("urls_new.csv",sep=',',encoding='utf-8',index=False)

In [ ]:
begin=time.time()
exclude=['twitter','glassdoor','youtube','wordpress','facebook','wikipedia','play.google','nasdaq']
allarcticles={}
logdcit={}
a=0
b=0
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
for g in f:
    start=time.time()
    print("link {}".format(a))
    if any (s in g[1] for s in exclude):
        print("excluded {} \n".format(g[1]))
        logdcit[a]=g[1] + "  **EXCLUDED"
    else:
        try:
            req = Request(g[1], headers=headers)
            with urlopen(req) as response:
                page = response.read()
            soup=BeautifulSoup(page, 'html.parser')
            ptag=soup.find_all('p')
            if len(ptag)<5:
                print("VERY LOW P TAGS, STUDY THIS - %s - %d" %(g[1], len(ptag)))
                logdcit[a]=g[1] + "  **VERY LOW P TAGS, STUDY THIS "
            text=[]
            for tag in ptag:
                text.append(tag.get_text())
            texta=str(' '.join(text)).replace('\n',' ')
            allarcticles[g[1]]=texta
            print("processed {} {} {}\n".format(g[1],b, "{:.2f}".format(time.time()-start)))
            b+=1
            logdcit[a]=g[1] + "  **PROCESSED"
        except http.client.IncompleteRead:
            print("INCOMPLETE READ - {} {}".format(g[1],"{:.2f}".format(time.time()-start)))
            logdcit[a]=g[1] + "  **INCOMPLETE READ"
        except WindowsError as e:
            print("WINDOWS ERROR {} {} {}".format(e,g[1],"{:.2f}".format(time.time()-start)))
            logdcit[a]=g[1] + "  **WINDOWS ERROR {}".format(e)
        except BaseException as error:
            print("SOME OTHER ERROR {} {}".format(link,"{:.2f}".format(time.time()-start)))
            logdcit[a]=g[1] + "  **SOME OTHER ERROR  {}".format(error)
            continue
        except timeout:
            print("LINK {} {} TIMED OUT \n".format(g[1],"{:.2f}".format(time.time()-start)))
            timedout.append(g[1])
            logdcit[a]=g[1]  + "  **TIMED OUT"
    a+=1
  
filename=datetime.datetime.now().strftime("%d_%m_%H_%M_%S")
df1=pd.DataFrame.from_dict(logdcit,orient='index')
df1.to_csv("LogFile_"+datetime.datetime.now().strftime("%d_%m_%H_%M_%S")+".csv",sep=',',encoding='utf-8')
df2=pd.DataFrame.from_dict(allarcticles,orient='index')
df2.to_csv("SUMMARIES_"+datetime.datetime.now().strftime("%d_%m_%H_%M_%S")+".csv",sep=',',encoding='utf-8')
#gc.collect
print("\n TOTAL TIME {}".format((time.time()-begin)/60))
    #try:    
    #    np.savetxt("accesserror.csv", accesserror, delimiter=",", fmt='%s')
    #except:
    #    print("cant save accesserror using np")


In [23]:
#EMAIL DATA
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

cursor.execute("Select ID,SearchResult from ProspectData WHERE emailupdates ='TRUE' AND ISPROCESSED='False'")
emaillist=[]
pid=[]
row = cursor.fetchone()
while row:
    #data=json.loads(str(row[1]))
    emaillist.append(str(row[1]))
    pid.append(row[0])
    row = cursor.fetchone()

#nd=[]
#ne=[]
#for c in zip(pid,emaillist):
#    if c[1] not in d:
#        nd.append(c[0])
#        ne.append(c[1])
##return zip(e,d)
#nf=zip(nd,ne)

In [29]:
len(emaillist)

12

In [31]:
print(emaillist[11])

EBRD logo [https://admin.govdelivery.com/attachments/fancy_images/UKEBRD/2015/10/630858/ebrd_large.jpg]
You have subscribed to News from the European Bank for Reconstruction and
Development (EBRD). We have just published this content.

EBRD PROVIDES FIRST LOCAL CURRENCY LOANS IN BELARUS
[HTTP://LINKS.GOVDELIVERY.COM:80/TRACK?TYPE=CLICK&ENID=ZWFZPTEMBXNPZD0MYXVPZD0MBWFPBGLUZ2LKPTIWMTGWODMWLJK0MJUZODQXJM1LC3NHZ2VPZD1NREITUFJELUJVTC0YMDE4MDGZMC45NDI1MZG0MSZKYXRHYMFZZWLKPTEWMDEMC2VYAWFSPTE3MJI2NZK4JMVTYWLSAWQ9BWFZDGVRYWXLCNRZQGDTYWLSLMNVBSZ1C2VYAWQ9BWFZDGVRYWXLCNRZQGDTYWLSLMNVBSZ0YXJNZXRPZD0MZMW9JMV4DHJHPU11BHRPDMFYAWF0ZULKPSYMJG==&&&100&&&HTTP://WWW.EBRD.COM/NEWS/2018/EBRD-PROVIDES-FIRST-LOCAL-CURRENCY-LOANS-IN-BELARUS.HTML]
30-08-2018 09:34 AM BST
CREDIT LINES TO LOCAL BANKS MTBANK AND BNB-BANK AND TO SUPPORT SME LENDING,
REDUCE FX-RELATED RISKS
FIND US ON
Facebook
[https://public.govdelivery.com/attachments/fancy_images/UKEBRD/2017/03/1275396/facebook-50_original.png]
[http://links.govd

In [40]:
import spacy

ModuleNotFoundError: No module named 'sputnik'

In [41]:
import sputnik

ModuleNotFoundError: No module named 'sputnik'